In [1]:
# Importing necessary libraries
import pandas as pd
from ast import literal_eval

In [2]:
# Load data
top_10_df = pd.read_csv('/Users/shounak/Documents/Cuvette/InterviewProj/IP2/nifty_50_top_10_selection_52 (1).csv')
percent_change_df = pd.read_csv('/Users/shounak/Documents/Cuvette/InterviewProj/IP2/nifty_50_accurate_percent_change_per_candle_52 (1).csv')

In [3]:
# Parse and merge
top_10_df['datetime'] = pd.to_datetime(top_10_df['datetime'])
top_10_df['top_10'] = top_10_df['top_10'].apply(literal_eval)
percent_change_df['datetime'] = pd.to_datetime(percent_change_df['datetime'])
merged_df = pd.merge(top_10_df, percent_change_df, on='datetime', how='inner')

In [4]:
# Initialising parameters
initial_investment = 1000
num_stocks = 10
initial_investment_per_stock = initial_investment / num_stocks

# Portfolio initialization
portfolio = {stock: initial_investment_per_stock for stock in merged_df.iloc[0]['top_10']}

# Track results
portfolio_values = []
average_changes = []

In [5]:
# Loop through each week
for _, row in merged_df.iterrows():
    current_top_10 = row['top_10']
    current_percent_changes = row.drop(['datetime', 'top_10'])

    # Updating portfolio value
    portfolio_value = 0
    for stock in list(portfolio.keys()):
        percent_change_column = f"{stock}_per_candle_percent_change_oc"
        if percent_change_column in current_percent_changes:
            percent_change = current_percent_changes[percent_change_column]
            portfolio[stock] *= (1 + percent_change / 100)
        portfolio_value += portfolio[stock]

    current_portfolio_stocks = set(portfolio.keys())
    new_portfolio_stocks = set(current_top_10)
    stocks_to_replace = current_portfolio_stocks - new_portfolio_stocks
    stocks_to_add = new_portfolio_stocks - current_portfolio_stocks

    if stocks_to_replace:
        avg_replace_value = sum(portfolio[stock] for stock in stocks_to_replace) / len(stocks_to_replace)
        for stock in stocks_to_replace:
            del portfolio[stock]
        for stock in stocks_to_add:
            portfolio[stock] = avg_replace_value

    # Track metrics
    average_changes.append(len(stocks_to_replace))
    portfolio_values.append(portfolio_value)


In [6]:
# Final calculations
average_no_of_changes = sum(average_changes) / len(average_changes)
total_portfolio_return = (portfolio_values[-1] - initial_investment) / initial_investment * 100

# Results
print(f"Total Portfolio Return: {total_portfolio_return:.2f}%")
print(f"Average Number of Changes per Week: {average_no_of_changes:.2f}")

Total Portfolio Return: -0.02%
Average Number of Changes per Week: 1.35
